In [3]:
import os
import time
from datetime import datetime
import operator
import re

import numpy as np
import numpy.ma as ma

import mmap

import pickle
import multiprocessing 

import csv

import gmplot
from os import path
from lxml import etree

In [2]:
curr_day = "211119"

# Searching through server logs - no particular need to do so
server_log_folder = "./server_logs/" + curr_day
server_folder_files = []

for r, d, f in os.walk(server_log_folder):
    for file in f:
        if '.txt' in file:
            server_folder_files.append(os.path.join(r, file))

In [260]:
# parse through the test_receive.txt file from server

rec_frames = []

test_receive = "server_logs/" + curr_day + "/test_receive.txt"
with open(test_receive) as search:
    for line in search:
        line = line.rstrip()  # remove '\n' at end of line
        if "receive frameID :" in line:
            frame_no = int(re.findall(r'receive frameID :(.+?), at time :', line)[0])
            rec_frames.append(frame_no)
                    
frames = np.zeros([len(rec_frames), 15])
""" 
The columns represent:
[0] : frame ID
[1] : frame size received
[2] : transmission delay
[3] : size of the result sent
[4] : time result was sent from server 
[5] : time taken to process the frame
[6] : time the frame was sent from device  
[7] : time a result was received on the device  
[8] : whether connection was 5G or 4G when frame sent from device
[9] : whether connection was 5G or 4G when result recieved on device
[10] : whether a handover occured between frame sent and result received on device
[11] : closest location: latitude
[12] : closest location: longitude
[13] : time server received the result
[14] : time client sent from device according to server log
"""

# reading the number of lines in the file
with open(test_receive) as f:
    num_lines_tr = sum(1 for _ in f)
    
contents_tr = open(test_receive, "r").readlines()
no_frame_parsed = 0 
for i in range(num_lines_tr):
    curr_line = contents_tr[i]
    
    if "receive frameID :" in curr_line:
        frame_no = int(re.findall(r'receive frameID :(.+?), at time :', curr_line)[0])
        frames[no_frame_parsed][0] = frame_no
        
        frame_size_rec = float(re.findall(r', has size:(.+?), transmission delay:', curr_line)[0])
        frames[no_frame_parsed][1] = frame_size_rec
        
        server_received_time = float(re.findall(r', at time : (.+?), sent out ', curr_line)[0])
        frames[no_frame_parsed][13] = server_received_time
    
        client_send_time = float(re.findall(r'from device at time: (.+?), has size:', curr_line)[0])
        frames[no_frame_parsed][14] = client_send_time
    
        no_frame_parsed += 1

In [261]:
# parse through the test_send.txt file from server

sent_frames = []

test_send = "server_logs/" + curr_day + "/test_send.txt"
with open(test_send) as search:
    for line in search:
        line = line.rstrip()  # remove '\n' at end of line
        if "send_result of frameID of:" in line:
            frame_no = int(re.findall(r'send_result of frameID of:(.+?)sent by observer at time:', line)[0])
            sent_frames.append(frame_no)
            
# reading the number of lines in the file
with open(test_send) as f:
    num_lines_ts = sum(1 for _ in f)
    
contents_ts = open(test_send, "r").readlines()
no_frame_parsed = 0 
for i in range(num_lines_ts):
    curr_line = contents_ts[i]
    
    if "send_result of frameID of:" in curr_line:
        frame_no = int(re.findall(r'send_result of frameID of:(.+?)sent by observer at time:', curr_line)[0])
        if frame_no == frames[no_frame_parsed][0]:
            frame_size_sent = curr_line.split(" ")[-1]
            frames[no_frame_parsed][3] = frame_size_sent
            
            frame_time_sent = float(re.findall(r'sent by observer at time:(.+?)whose size is:', curr_line)[0])
            frames[no_frame_parsed][4] = frame_time_sent
            
        no_frame_parsed += 1

In [262]:
# parse through the test_process.txt file from server

process_frames = []

test_process = "server_logs/" + curr_day + "/test_process.txt"
with open(test_process) as search:
    for line in search:
        line = line.rstrip()  # remove '\n' at end of line
        if "time_process_pic of frameid of:" in line:
            frame_no = int(re.findall(r'time_process_pic of frameid of:(.+?)takes: ', line)[0])
            process_frames.append(frame_no)
            
# reading the number of lines in the file
with open(test_process) as f:
    num_lines_tp = sum(1 for _ in f)
    
contents_tp = open(test_process, "r").readlines()
no_frame_parsed = 0 
for i in range(num_lines_tp):
    curr_line = contents_tp[i]
    
    if "time_process_pic of frameid of:" in curr_line:
        frame_no = int(re.findall(r'time_process_pic of frameid of:(.+?)takes: ', curr_line)[0])
        if frame_no == frames[no_frame_parsed][0]:
            frame_process_time = re.findall(r'takes:(.+?)milliseconds', curr_line)[0]
            frame_process_time = float(re.findall(r"'(.+?)'", frame_process_time)[0])
            frames[no_frame_parsed][5] = frame_process_time
            
        no_frame_parsed += 1

In [263]:
# parse through client send.txt and receive.txt

client_log_folder = "./device_logs/"

# send.txt
send_clf = client_log_folder + "send.txt"
with open(send_clf) as f:
    num_lines_sclf = sum(1 for _ in f)

sent_frames = []
contents_scl = open(send_clf, "r").readlines()
for i in range(num_lines_sclf):
    curr_line_cscl = contents_scl[i].rstrip()
    clcscl_split = curr_line_cscl.split(",")
    
    sent_frame_id = int(clcscl_split[0])
    sent_time = float(clcscl_split[1])
    
    sent_frames.append([sent_frame_id, sent_time])
    
# receive.txt
receive_clf = client_log_folder + "receive.txt"
with open(receive_clf) as f:
    num_lines_rclf = sum(1 for _ in f)

received_frames = []
contents_rcl = open(receive_clf, "r").readlines()
for i in range(num_lines_rclf):
    curr_line_crcl = contents_rcl[i].rstrip()
    clcrcl_split = curr_line_crcl.split(",")
    
    received_frame_id = int(clcrcl_split[0])
    received_time = float(clcrcl_split[1])
    
    received_frames.append([received_frame_id, received_time])

In [264]:
# matching up the times in sent_frames list and the server received frames array

server_received = list(frames[:,14])
csf = [item[1] for item in sent_frames]

matched_frames = []
for i in range(len(server_received)):
    curr_sr_time = server_received[i]
    match_s_c = np.where(curr_sr_time == csf)[0]
    
    if not match_s_c.size == 0:
        loc_match = int(match_s_c[0])
        
        # place time into frames array, even if the same
        frames[i][6] = curr_sr_time

In [265]:
# matching up the server sent and client received logs

crf = [item[1] for item in received_frames]
csf_ids = [item[0] for item in sent_frames]

# match up client send and receive arrays
for i in range(len(crf)):
    curr_crf = crf[i]
    curr_cr_id = received_frames[i][0]
    
    match_s_r = np.where(curr_cr_id == np.asarray(csf_ids))[0]
    for j in range(len(match_s_r)):
        curr_match = match_s_r[j]
        csf_time = sent_frames[curr_match][1]
        csf_id = sent_frames[curr_match][0]
        diff = curr_crf - csf_time
        if 0 < diff < 1000:
            # finding location from device timestamp of when sent from client 
            frames_locs = np.where(frames[:,14] == csf_time)[0]
            
            if not frames_locs.size == 0:
                # select first instance of the frame (if it is repeated)
                frame_loc = frames_locs[0]
                frames[frame_loc][7] = curr_crf

## Parsing network logs for 5GTN latency results

In [5]:
# search through network logs and extract the data from PCAP file which is now
# a text file - export/conversion done with Wireshark

network_log_folder = "./network_logs/"
network_folder_files = []

for r, d, f in os.walk(network_log_folder):
    for file in f:
        if '.pcap' in file:
            network_folder_files.append(os.path.join(r, file))

network_folder_file = network_folder_files[-1]
split_nff = network_folder_file.split("/")[-1].split(".")[0]
new_txt = network_log_folder + split_nff + ".txt"

IndexError: list index out of range

In [3]:
# reading through the network log and selecting out packets and associate data

network_dict = {
    "19" : 5, # 5G,
    "20" : 4, # 4G/LTE
    "182" : 0 # unknown
}

# reading the number of lines in the file
with open(new_txt) as f:
    num_lines_nl = sum(1 for _ in f)

packets = []       
network_log = open(new_txt, "r").readlines()
for i in range(num_lines_nl):
    curr_line = network_log[i]
    #print(curr_line)
    
    if "bytes on wire" in curr_line:
        cl_split = curr_line.split(" ")
        packet_id = cl_split[1].replace(":", "")
        
        # variable changes to 1 if a handover occurs 
        handover_event = 0 
    if "Epoch Time:" in curr_line:
        packet_time = re.findall(r'Epoch Time:(.+?)seconds', curr_line)[0]
    if "E-RABModificationIndication" in curr_line:
        handover_event = 1 
    if "transportLayerAddress(IPv4):" in curr_line:
        cl_split = curr_line.split(" ")
        network_ip = cl_split[-1].strip()
        
        nip_split = network_ip.split(".")[-1]
        network_type = network_dict[nip_split]
        
        packet_data = [packet_id, packet_time, handover_event, network_type]
        packets.append(packet_data)

NameError: name 'new_txt' is not defined

In [15]:
# creating array of periods to show when network is 5G or 4G

network_periods = []

begin_time = 0 # assume time begun at 0 
network_type = 5 # network began at 5G
for i in range(len(packets)):
    curr_packet = packets[i]
    handover_time = curr_packet[1]
    
    period_data = [float(begin_time), float(handover_time), network_type]
    network_periods.append(period_data)
    begin_time = handover_time
    if i != 0:
        network_type = curr_packet[3]

In [16]:
# match packets list to the frames numpy array

for i in range(len(frames)):
    curr_frame = frames[i]
    device_sent_time = curr_frame[6] / 1000
    device_received_time = curr_frame[7] / 1000
    
    handover_during = []

    for j in range(len(network_periods)):
        curr_period = network_periods[j]
        cp_begin = curr_period[0]
        cp_end = curr_period[1]
        cp_type = curr_period[2]
        
        if cp_begin < device_sent_time < cp_end: 
            frames[i][8] = cp_type
            #print("network at sent: ", cp_type, ", ", device_sent_time, ", ", cp_begin, " ", cp_end)
            handover_during.append(cp_type)
        if cp_begin < device_received_time < cp_end: 
            frames[i][9] = cp_type
            #print("network at received: ", cp_type, ", ", device_received_time, ", ", cp_begin, " ", cp_end)
            handover_during.append(cp_type)

    if np.sum(handover_during) == 9:
        # if constant network leave value of 0
        # if 5G>4G handover assign value of 1
        # if 4G>5G handover assign value of 2
        
        if handover_during[0] == 5:
            frames[i][10] = 1
        else:
            frames[i][10] = 2
            
        #print(cp_begin, cp_end, device_sent_time, device_received_time)


In [17]:
# analysing the handovers

# changing from 5G to 4G, and 4G to 5G
five_four = []
four_five = []

# duration spent on each network
five = []
four = []

network_type = 5
for i in range(len(network_periods)):
    curr_period = network_periods[i]
    new_network_type = curr_period[2]
    
    nnt_begin = curr_period[0]
    nnt_end = curr_period[1]
    
    dur_on_type = nnt_end - nnt_begin
    if new_network_type == 5:
        five.append(dur_on_type)
    if new_network_type == 4:
        four.append(dur_on_type)
    
    if new_network_type == 0:
        new_network_type = network_type

    if network_type != new_network_type:
        #print("changing from ", network_type, " to ", new_network_type)
        
        if network_type == 5:
            five_four.append(nnt_begin)
        else:
            four_five.append(nnt_begin)
    
    network_type = new_network_type
    
print("Changed from 5G to 4G:", len(five_four), " times, and 4G to 5G:", 
      len(four_five), " times")

five = five[1:-1]
print("Median time spent on 5G network before change is:", np.median(five), "(ms), minimum is:", 
      np.min(five), "(ms), and maximum is:", np.max(five), "(ms)")

four = four[1:-1]
print("Median time spent on 4G network before change is:", np.median(four), "(ms), minimum is:", 
      np.min(four), "(ms), and maximum is:", np.max(four), "(ms)")


Changed from 5G to 4G: 85  times, and 4G to 5G: 85  times
Median time spent on 5G network before change is: 8.995527982711792 (ms), minimum is: 0.0460050106048584 (ms), and maximum is: 504.08662009239197 (ms)
Median time spent on 4G network before change is: 1.092229962348938 (ms), minimum is: 0.0918588638305664 (ms), and maximum is: 679.1013910770416 (ms)


In [66]:
# calculating the data transfer times for the whole experimental period

print("In total,", len(frames), "frames were sent, experiment lasted for a total of",
     (frames[-2][7]-frames[0][6])/1000/60, "minutes")
print("Changed from 5G to 4G:", len(five_four), " times, and 4G to 5G:", len(four_five), 
      " times - totalling", len(five_four)+len(four_five), "handovers")
print("Phone detected", len(location_changes), "location changes")

# did handover occur
handover_question = frames[:,10]
network_cs = frames[:,8] # network when client sent
network_cr = frames[:,9] # network when client received

# client processing time
client_sent_time = frames[:,6]
client_received_time = frames[:,7]
client_tot_time = client_received_time - client_sent_time

# server processing time
server_process_tot = frames[:,5]

data_transfer_time = client_tot_time - server_process_tot
overall_med_dtt = np.median(data_transfer_time)
print("Overall median data transfer for whole experiment:", overall_med_dtt, "(ms)")

###

when_5g = []
when_4g = []

when_5g_4g = [] # 5G to 4G
when_4g_5g = [] # 4G to 5G

server_processing_times = []
client_time_gap = []

for i in range(len(frames)):
    curr_frame = frames[i]
    curr_network_type_cs = curr_frame[8]
    curr_network_type_cr = curr_frame[9]
    
    curr_cst = curr_frame[6]
    curr_crt = curr_frame[7]
    curr_tott = curr_crt - curr_cst
    #print(curr_tott, curr_crt, curr_cst)
        
    curr_spt = curr_frame[5]
    server_processing_times.append(curr_spt)
    
    # data transfer time
    curr_dtt = curr_tott - curr_spt
    #print(curr_dtt, curr_tott, curr_spt)
    
    curr_ncs = curr_frame[8]
    curr_ncr = curr_frame[9]

    if curr_network_type_cs == curr_network_type_cr:
        # data transfer times when on the same network type continually from
        # frame sent to receive
        if curr_network_type_cs == 5:
            if 0 < curr_dtt < 5000:
                when_5g.append(curr_dtt)
        if curr_network_type_cs == 4:
            if 0 < curr_dtt < 5000:
                when_4g.append(curr_dtt)
    else:
        # if handover occurs betwene sending and receiving a frame
        if curr_ncs == 5:
            # if from 5G to 4G handover
            if 0 < curr_dtt < 5000:
                client_time_gap.append(curr_dtt)
                when_5g_4g.append(curr_dtt)
        if curr_ncs == 4:
            if 0 < curr_dtt < 5000:
                client_time_gap.append(curr_dtt)
                when_4g_5g.append(curr_dtt)

avg_spt = np.average(server_processing_times)
std_spt = np.std(server_processing_times)
print("Average server processing times:", avg_spt, "+-", std_spt, "(ms)")

avg_ctg = np.average(client_time_gap)
std_ctg = np.std(client_time_gap)
print("Average client time gap:", avg_ctg, "+-", std_ctg, "(ms)")
            
med_5g = np.median(when_5g)
print(np.max(when_5g))
print("Median data transfer time when continual 5G connection:", med_5g, "(ms)")

med_4g = np.median(when_4g)
print("Median data transfer time when continual 4G connection:", med_4g, "(ms)")

med_5g_4g = np.median(when_5g_4g)
print("Median data transfer time when handover from 5G to 4G connection:", med_5g_4g, "(ms)")

med_4g_5g = np.median(when_4g_5g)
print("Median data transfer time when handover from 4G to 5G connection:", med_4g_5g, "(ms)")

frames_size = frames[:,1]
avg_fs = np.average(frames_size) # in bits
afs_bytes = avg_fs/8
print("Average frame size received from client on server", avg_fs)

client_sent_orig = frames[:,6]
client_sent_shift = np.roll(client_sent_orig, -1)
cs_time_diff = client_sent_shift - client_sent_orig
cs_time_diff = cs_time_diff[cs_time_diff > 0]
cstd_med = np.median(cs_time_diff)
print("Median time between frame sent from client:", cstd_med, "(ms)")

In total, 11842 frames were sent, experiment lasted for a total of 56.19185 minutes
Changed from 5G to 4G: 85  times, and 4G to 5G: 85  times - totalling 170 handovers
Phone detected 2803 location changes
Overall median data transfer for whole experiment: 748.1795999999999 (ms)
Average server processing times: 18.5334517522378 +- 10.15219806839911 (ms)
Average client time gap: 793.3137859649123 +- 666.0245725576414 (ms)
4985.0989
Median data transfer time when continual 5G connection: 246.8162 (ms)
Median data transfer time when continual 4G connection: 387.241 (ms)
Median data transfer time when handover from 5G to 4G connection: 286.5229 (ms)
Median data transfer time when handover from 4G to 5G connection: 878.9599499999999 (ms)
Average frame size received from client on server 40257.83642965715
Median time between frame sent from client: 37.0 (ms)


In [1]:
# plotting location data onto Google Maps

# load 5GTN data
kml_file = path.join("5GTN coverage.kml")
kml_parsed = etree.parse(kml_file)
kml_to_string = etree.tostring(kml_parsed, pretty_print=True).decode("utf-8") 

#print(kml_to_string)

lat_list = [float(item[0]) for item in location_changes]
long_list = [float(item[1]) for item in location_changes]

uni_map = gmplot.GoogleMapPlotter(lat_list[0], long_list[0], 17) 

#uni_map.scatter(lat_list, long_list, '#FFFFFF', size=1, marker=False) 
uni_map.plot(lat_list, long_list, 'red', edge_width=2.5) 

# comparing the times when in certain network types, and the locations
loc_5g = []
loc_4g = []
for i in range(len(network_periods)):
    curr_period = network_periods[i]
    cp_begin = curr_period[0]
    cp_end = curr_period[1]
    cp_type = curr_period[2]
    for j in range(len(location_changes)):
        curr_loc = location_changes[j]
        cl_time = float(curr_loc[2]) / 1000
        curr_lat = float(curr_loc[0])
        curr_long = float(curr_loc[1])
        if cp_begin < cl_time < cp_end:
            if cp_type == 5:
                loc_5g.append([curr_lat, curr_long])
            if cp_type == 4:
                loc_4g.append([curr_lat, curr_long])
                
# plotting 5G locations
fiveg_lat_list = [float(item[0]) for item in loc_5g]
fiveg_long_list = [float(item[1]) for item in loc_5g]
uni_map.scatter(fiveg_lat_list, fiveg_long_list, 'orange', size=1, marker=False) 

# plotting 4G locations
fourg_lat_list = [float(item[0]) for item in loc_4g]
fourg_long_list = [float(item[1]) for item in loc_4g]
uni_map.scatter(fourg_lat_list, fourg_long_list, 'blue', size=1, marker=False) 

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

loc_changes_times = [float(item[2])/1000 for item in location_changes]

# going through the times when on a new network type and plot
fiveg_fourg = np.zeros([len(five_four), 2])
for i in range(len(five_four)):
    curr_five = five_four[i]
    nearest_time = find_nearest(loc_changes_times, curr_five)
    nt_index = nearest_time[1]
    curr_lat = float(location_changes[nt_index][0])
    curr_long = float(location_changes[nt_index][1])
    
    fiveg_fourg[i][0] = curr_lat
    fiveg_fourg[i][1] = curr_long
    
for i in range(len(fiveg_fourg)):
    curr_llat = fiveg_fourg[i][0]
    curr_llong = fiveg_fourg[i][1]
    uni_map.marker(curr_llat, curr_llong, 'orangered') 
#uni_map.marker(fiveg_fourg[:,0], fiveg_fourg[:,1], 'purple', size=1, marker=False)

fourg_fiveg = np.zeros([len(four_five), 2])
for i in range(len(four_five)):
    curr_four = four_five[i]
    nearest_time = find_nearest(loc_changes_times, curr_four)
    nt_index = nearest_time[1]
    curr_lat = float(location_changes[nt_index][0])
    curr_long = float(location_changes[nt_index][1])
    
    fourg_fiveg[i][0] = curr_lat
    fourg_fiveg[i][1] = curr_long

for i in range(len(fourg_fiveg)):
    curr_llat = fourg_fiveg[i][0]
    curr_llong = fourg_fiveg[i][1]
    uni_map.marker(curr_llat, curr_llong, 'steelblue') 
#uni_map.marker(fourg_fiveg[:,0], fourg_fiveg[:,1], 'green', size=1, marker=False)
    
# manually plotting 5G and LTE modems
uni_map.marker(65.0578097,25.4687127, 'orange') # 5G

# these coords are the wrong way round - long then lat
lte_modems = [
    [25.4687287,65.0576514],
    [25.4692223,65.0581491],
    [25.4694208,65.0581717],
    [25.4657354,65.0586965],
    [25.4662504,65.0588051],
    [25.4688253,65.0590946],
    [25.4692277,65.0578708],
    [25.4693081,65.0581672]
]

lte_lats = [float(item[1]) for item in lte_modems]
lte_longs = [float(item[0]) for item in lte_modems]
for i in range(len(lte_lats)):
    curr_llat = lte_lats[i]
    curr_llong = lte_longs[i]
    uni_map.marker(curr_llat, curr_llong, 'blue') # 4G/LTE

uni_map.draw("location_changes.html") 

NameError: name 'path' is not defined

## Parsing results for just on WiFi

In [294]:
# removing frames which are sent from client, but no results received back

frames_client_received = frames[:,7]

# creating a mask for elements which are 0
fcr_mask = frames_client_received > 0
fcr_mask = fcr_mask.astype(np.int)

# cycle through the mask and remove the rows from the frames array
fcrm_zeros = np.where(fcr_mask == 0)
fcrm_zeros = np.flip(fcrm_zeros)

useful_frames = frames
for i in range(len(fcrm_zeros)):
    curr_itr = fcrm_zeros[i]
    useful_frames = np.delete(useful_frames, curr_itr, axis=0)

In [314]:
# calculating data transfer times

print("In total,", len(useful_frames), 
      "frames were sent from client, experiment lasted for a total of",
     (useful_frames[-2][7]-useful_frames[0][6])/1000/60, "minutes")

# client processing time
client_sent_time = useful_frames[:,6]
client_received_time = useful_frames[:,7]
client_tot_time = client_received_time - client_sent_time

ctt_med = np.median(client_tot_time)
ctt_std = np.std(client_tot_time)
print("Median time for client sending frame and receiving a result", ctt_med, 
      "+-", ctt_std, "(ms)")

# server processing time
server_process_tot = useful_frames[:,5]

spt_med = np.median(server_process_tot)
spt_std = np.std(server_process_tot)
print("Median time for server processing", spt_med, "+-", spt_std, "(ms)")

data_transfer_time = client_tot_time - server_process_tot
overall_med_dtt = np.median(data_transfer_time)
overall_std_dtt = np.std(data_transfer_time)
print("Overall median data transfer for whole experiment:", overall_med_dtt, 
      "+-", overall_std_dtt, "(ms)")

In total, 1603 frames were sent from client, experiment lasted for a total of 2.6630333333333334 minutes
Median time for client sending frame, and receiving a result 169.0 +- 62.669782624026325 (ms)
Median time for server processing 14.8909 +- 10.4467702472718 (ms)
Overall median data transfer for whole experiment: 149.3738 +- 64.29969014377126 (ms)
